<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Machine Learning with PyTorch for Developers</h1>
<h1>Machine Learning Overview</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [ ]:
from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torch.nn.functional as F

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

autopep8  1.5
numpy     1.18.1
json      2.0.9
pandas    1.0.1
watermark 2.0.2
Mon May 11 2020 

CPython 3.7.3
IPython 6.2.1

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 19.4.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit
Git hash   : f06693032ded22cc83a4db4c4a7a590c1fa4bc94


Load default figure style

In [3]:
plt.style.use('./d4sci.mplstyle')

# Start

In [ ]:
class PCA(nn.Module):
    def __init__(self, n_components):
        super().__init__()
        self.n_components = n_components
        self.mean = None
        self.components = None
        self.explained_variance = None
        
    def fit(self, X):
        """
        Fit the PCA model
        X: input tensor of shape [n_samples, n_features]
        """
        # Center the data
        self.mean = torch.mean(X, dim=0)
        X_centered = X - self.mean
        
        # Compute covariance matrix
        N = X.size(0)
        cov = torch.mm(X_centered.t(), X_centered) / (N - 1)
        
        # Compute eigenvalues and eigenvectors
        eigenvalues, eigenvectors = torch.linalg.eigh(cov)
        
        # Sort eigenvalues and eigenvectors in descending order
        idx = torch.argsort(eigenvalues, descending=True)
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]
        
        # Store principal components and explained variance
        self.components = eigenvectors[:, :self.n_components]
        self.explained_variance = eigenvalues[:self.n_components]
        
        # Calculate explained variance ratio
        self.explained_variance_ratio = eigenvalues[:self.n_components] / torch.sum(eigenvalues)
        
        return self
    
    def transform(self, X):
        """
        Transform data
        X: input tensor of shape [n_samples, n_features]
        """
        X_centered = X - self.mean
        return torch.mm(X_centered, self.components)
    
    def fit_transform(self, X):
        """
        Fit and transform in one step
        """
        return self.fit(X).transform(X)

# Example usage (commented out):
# X = torch.randn(100, 10)  # 100 samples, 10 features
# pca = PCA(n_components=2)
# X_transformed = pca.fit_transform(X)
# print(f"Explained variance ratio: {pca.explained_variance_ratio}")

In [ ]:
# K-Means Implementation in PyTorch
# We assume torch is already imported from previous cells.

import torch.nn as nn

class KMeans(nn.Module):
    def __init__(self, n_clusters, max_iter=100):
        super().__init__()
        self.n_clusters = n_clusters
        self.max_iter = max_iter

    def forward(self, data):
        """
        data: A tensor of shape [N, D] where N is the number of samples, and D is the number of features.
        Returns:
        - cluster_assignments: Tensor of shape [N], each element is the cluster index for each data point.
        - centroids: Tensor of shape [n_clusters, D], the final cluster centers.
        """
        
        # 1. Initialize cluster centers by picking random points from data
        indices = torch.randperm(data.size(0))[:self.n_clusters]
        centroids = data[indices]

        for _ in range(self.max_iter):
            # 2. Compute distances between data points and centroids
            #    We use broadcasting to find squared distances to each centroid.
            distances = torch.cdist(data, centroids, p=2)

            # 3. Assign each data point to the closest centroid
            cluster_assignments = distances.argmin(dim=1)

            # 4. Update the centroids based on current cluster assignments
            new_centroids = []
            for k in range(self.n_clusters):
                cluster_points = data[cluster_assignments == k]
                if len(cluster_points) > 0:
                    new_centroids.append(cluster_points.mean(dim=0))
                else:
                    # If no points are assigned to a cluster, keep the old centroid
                    new_centroids.append(centroids[k])
            new_centroids = torch.stack(new_centroids)

            # 5. Check if centroids have changed significantly
            if torch.allclose(centroids, new_centroids, atol=1e-4):
                break
            centroids = new_centroids

        return cluster_assignments, centroids

# Usage Example (uncomment to run):
# data_tensor = torch.randn(100, 2)
# kmeans = KMeans(n_clusters=3, max_iter=10)
# assignments, final_centroids = kmeans(data_tensor)
# print("Cluster Assignments:", assignments)
# print("Final Centroids:", final_centroids)

In [ ]:
 Introduction to PyTorch basics
print("PyTorch Introduction:")a

# 1. Creating a simple Neural Network Layer
linear = torch.nn.Linear(in_features=2, out_features=1)
print(f"\nNeural Network Layer:\n{linear}")

# 2. Define a simple optimization problem
# Create random input data
X = torch.randn(10, 2)  # 10 samples, 2 features
y = torch.randn(10, 1)  # 10 targets

# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# 3. Simple training loop example
print("\nTraining Loop Example:")
for epoch in range(5):
    # Forward pass
    y_pred = linear(X)
    loss = criterion(y_pred, y)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 1 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

print("\nOptimization components:")
print(f"Optimizer: {type(optimizer).__name__}")
print(f"Loss Function: {type(criterion).__name__}")

<div style="width: 100%; overflow: hidden;">
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</div>